| version | commentaire |
| --- | --- |
| v0r1 | version initiale |
| v0rN |  |


# Forum des métiers - Allocation de slots selon préférence élèves prioritaire

## Introduction

Ce notebook permet d'effectuer des allocations individuelles à des slots de visite de représentants des métiers

Contraintes :

- pas d'élève sans affectation de métier pour un créneau (timeslot) donné,
- le moins possible de métier (job) sans affectation de groupe pour un timeslot donné,
- nombre de souhaits par élève =< timeslots,
- il peut y avoir n représentants pour un métier donné.

=> le nombre de métiers doit être plus grand que le nombre de groupes d'élèves pour que chaque groupe d'élèves visite un métier pendant un créneau mais pas trop non plus pour minimiser les métiers sans groupe affecté pendant un créneau.


La conversion du contenu d'un fichier Excel au formal JSON peut être effectuée sur le site suivant :
https://tableconvert.com/excel-to-json

## Importations des modules

In [1]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import math
import random
from json import loads, dumps
from numpy.random import default_rng
from dataclasses import dataclass
from collections import defaultdict

In [2]:
from modules.typedef import *
from modules.q_utils import *
from modules.student_utils import scrap_student_choice, get_copy_students, get_next_valid_student_choice, scrap_single_student_choice, convert_to_student_timetable
from modules.timetable_utils import timetable_student_view, get_timetable_intervenants
from modules.evaluation import get_evaluation

In [4]:
# importation d'un échantillon de données pour les tests unitaires
sample_class_32 = pd.read_json('sample-3e2-new.json')
result = sample_class_32.to_json(orient="index")
parsed = loads(result)
sample_class_32 = list(parsed.values())


In [5]:
type(sample_class_32[0])

dict

In [6]:
intervenants_data = pd.read_json('intervenants_data_sample.json')

In [7]:
intervenants_data

,0,1,2,3,4,5
0,None,None,None,None,None,None
1,"{'id': 10, 'batch_size': 4}","{'id': 11, 'batch_size': 4}","{'id': 12, 'batch_size': 4}",None,None,None
2,"{'id': 20, 'batch_size': 4}","{'id': 21, 'batch_size': 4}",None,None,None,None
3,"{'id': 30, 'batch_size': 4}","{'id': 31, 'batch_size': 4}",None,None,None,None
4,"{'id': 40, 'batch_size': 4}",None,None,None,None,None
5,"{'id': 50, 'batch_size': 4}","{'id': 51, 'batch_size': 4}","{'id': 52, 'batch_size': 4}","{'id': 53, 'batch_size': 4}","{'id': 54, 'batch_size': 4}","{'id': 55, 'batch_size': 4}"


In [8]:
NB_JOBS = intervenants_data.shape[0]
NB_JOBS

6

# Allocations des élèves aux timeslots

Un timeslot est un créneau horaire durant lequel différents batch d'élèves vont visiter un (et un seul) intervenant de métier (job).

Un timeslot est représenté par un vecteur dont chaque index est celui d'un intervenant de métier et la valeur à cet index est une liste d'élèves (batch pour ce slot).
La taille du batch peut être différente pour chaque représentant dans un même timeslot

```
intervenant 0         1    2    3   4   
slot 0 [ [ student ], .. , .., .., .. ]
slot 1 [    ..      , .. , .., .., .. ]
slot 2 [    ..      , .. , .., .., .. ]
  :
```

La fonction d'entrée de l'allocation des timeslots est `allocate_slots`.

## Q-intervenant

Avant de constituer les batch d'un timeslot, on place les élèves dans des files d'attente par intervenant représentant la catégorie de métier.

Un élève est placé dans une file d'attente en fonction de ses préférences de métier (non encore visitées) et par ordre de préférence.

Un élève ne peut être placé dans une file d'attente que si celle-ci n'est pas déjà pleine (pour un intervenant donné).

Si toutes les files d'attente des représentants d'un métier sont pleines, on considère le métier non encore visité suivant de l'élève et on retente une affectation dans une file d'attente correspondante.

Si l'élève ne peut toujours pas être placé dans une file d'attente selon ses préférences métier, on le place dans une file d'attente encore libre d'un intervenant que cet élève n'a pas encore visité.

Exemple :

Pour un élève qui a 5 choix dont le premier a déjà été visité dans un timeslot antérieur, pour la file d'attente (Q-intervenant) du timeslot en constitution, on place l'élève dans la file d'attente d'un intervenant du métier 3 si la file de cet intervenant n'est pas encore pleine (selon BATCH_SIZE[intervenant]).

```
{ "id" : 3100, wishes : [ -1, 3, 13, 8, 5 ]}

Q[<intervenant(3)>].append(3100)
````

Si un métier a plusieurs intervenants, on place l'élève dans la file d'attente la plus courte.

Si toutes les files d'attente des intervenants du métier No 3 sont pleines, on reprend la tentative d'affectation avec le métier 13.

Si malgré tout l'élève ne peut pas être placé dans une file d'attente, on le place dans une file d'attente d'un intervenant que cet élève n'a pas encore visité dans un précédent timeslot. 

In [9]:
# test
test_students = [ 
    { "id" : 3100, "wishes" : [ 4,3,2,1,0 ] },
    { "id" : 3101, "wishes" : [ 0,1,2,3,4 ] },
    { "id" : 3102, "wishes" : [ 2,1,0,4,3 ] }
]
scrap_student_choice(test_students, 3101, 1)

[{'id': 3100, 'wishes': [4, 3, 2, 1, 0]},
 {'id': 3101, 'wishes': [0, -1, 2, 3, 4]},
 {'id': 3102, 'wishes': [2, 1, 0, 4, 3]}]

## Fonctions Allocate slots - std driven

In [10]:
def q_students_pass1(n_slot:int, q_intervenants:Q_Intervenants, students:list[Student])->Tuple[Q_Intervenants, list[Student]]:

    result_students = get_copy_students(students)

    #result_students = [ { **student, **{ 'bck_wishes': list(student['wishes']) }}  for student in result_students ]

    remain_students = result_students
    
    while(len(remain_students)>0):
        tmp_students = remain_students
        remain_students = []

        for student in tmp_students:
            # get next job
            next_job = get_next_valid_student_choice(tmp_students, student['id'])
            if next_job == None :
                pass
            else:
                # q available ?
                next_q = get_next_avail_q(q_intervenants, student, next_job)

                student = scrap_single_student_choice(student, next_job)
                if next_q != None:
                    #q_intervenants = q_single_student(q_intervenants, student['id'], next_job, next_q)
                    q_intervenants = push_student_in_q(q_intervenants, next_job, next_q, student['id'])
                    student['itv_visited'].append((n_slot, (next_job,next_q)))
                else:
                    remain_students.append(student)

    result_students = list([ { 'id':s1['id'], 'wishes':list(s1['wishes']), 'itv_visited':list(s2['itv_visited']) } for s1, s2 in zip(students,result_students) ])
    for student in result_students:
        l_job_visited = [ j for (_, (j, _)) in student['itv_visited']]
        for j in l_job_visited:
            student = scrap_single_student_choice(student, j)

        # if student did not get a job visit for this slot, insert a (slot, None) visit in student data
        l_slot_fullfilled = [ slot for (slot, _) in student['itv_visited'] ]
        if not n_slot in l_slot_fullfilled:
            student['itv_visited'].append((n_slot, (-1,0)))

    
    return (q_intervenants,result_students)

In [11]:
def get_timetable_student_driven(nb_slots:int, students:list[Student], intervenants_data:pd.DataFrame)->Timetable:
    result_qs = []
    for slot in range(nb_slots):
        q_intervenants = init_q_intervenants(intervenants_data)
        result_q, students = q_students_pass1(slot, q_intervenants, students)
        result_qs.append(result_q)
    
    #timetable = []
    #for n in range(nb_slots):
    #    d = []
    #    for student in students:
    #        d.append([ student['id'], (student['itv_visited'][n][1],student['itv_visited'][n][2])  if n < len(student['itv_visited']) else None])
    #    timetable.append(d)
    
    return convert_to_student_timetable(nb_slots, students), students

In [12]:
# test : q_students_pass1
test_students = [ 
    { "id" : 3100, "wishes" : [ 2,1,3,4,5 ], 'itv_visited' : [] }, 
    { "id" : 3101, "wishes" : [ 2,1,3,5,4 ], 'itv_visited' : []  }, 
    { "id" : 3102, "wishes" : [ 2,1,5,4,3 ], 'itv_visited' : []  }, 
    { "id" : 3103, "wishes" : [ 2,1,5,4,3 ], 'itv_visited' : []  },
    { "id" : 3104, "wishes" : [ 2,1,4,5,3 ], 'itv_visited' : []  }, 
    { "id" : 3105, "wishes" : [ 2,1,5,3,4 ], 'itv_visited' : []  }, 
    { "id" : 3106, "wishes" : [ 2,1,3,4,5 ], 'itv_visited' : []  }, 
    { "id" : 3107, "wishes" : [ 2,1,3,4,5 ], 'itv_visited' : []  },
    { "id" : 3108, "wishes" : [ 2,1,5,3,4 ], 'itv_visited' : []  }, 
    { "id" : 3109, "wishes" : [ 2,1,5,4,3 ], 'itv_visited' : []  }, 
    { "id" : 3110, "wishes" : [ 2,4,5,1,3 ], 'itv_visited' : []  }, 
    { "id" : 3111, "wishes" : [ 2,4,5,1,3 ], 'itv_visited' : []  },
    { "id" : 3112, "wishes" : [ 4,2,1,3,5 ], 'itv_visited' : []  }
]
NB_SLOTS = 5

timetable, test_students = get_timetable_student_driven(NB_SLOTS, test_students, intervenants_data)
print(pd.DataFrame(timetable))
for student in test_students:
    print(student)

                    0                    1                    2   \
0  (3100, (0, (2, 0)))  (3101, (0, (2, 0)))  (3102, (0, (2, 0)))   
1  (3100, (1, (1, 0)))  (3101, (1, (1, 0)))  (3102, (1, (1, 0)))   
2  (3100, (2, (3, 0)))  (3101, (2, (3, 0)))  (3102, (2, (5, 0)))   
3  (3100, (3, (4, 0)))  (3101, (3, (5, 0)))  (3102, (3, (4, 0)))   
4  (3100, (4, (5, 0)))  (3101, (4, (4, 0)))  (3102, (4, (3, 0)))   

                    3                    4                    5   \
0  (3103, (0, (2, 0)))  (3104, (0, (2, 1)))  (3105, (0, (2, 1)))   
1  (3103, (1, (1, 0)))  (3104, (1, (1, 1)))  (3105, (1, (1, 1)))   
2  (3103, (2, (5, 0)))  (3104, (2, (4, 0)))  (3105, (2, (5, 0)))   
3  (3103, (3, (4, 0)))  (3104, (3, (5, 0)))  (3105, (3, (3, 0)))   
4  (3103, (4, (3, 0)))  (3104, (4, (3, 0)))  (3105, (4, (4, 0)))   

                    6                    7                    8   \
0  (3106, (0, (2, 1)))  (3107, (0, (2, 1)))  (3108, (0, (1, 0)))   
1  (3106, (1, (1, 1)))  (3107, (1, (1, 1)))  (

In [13]:
timetable_student_view(timetable)

defaultdict(<class 'list'>, {3100: ['2-0', '1-0', '3-0', '4-0', '5-0'], 3101: ['2-0', '1-0', '3-0', '5-0', '4-0'], 3102: ['2-0', '1-0', '5-0', '4-0', '3-0'], 3103: ['2-0', '1-0', '5-0', '4-0', '3-0'], 3104: ['2-1', '1-1', '4-0', '5-0', '3-0'], 3105: ['2-1', '1-1', '5-0', '3-0', '4-0'], 3106: ['2-1', '1-1', '3-0', '4-0', '5-0'], 3107: ['2-1', '1-1', '3-0', '5-0', '4-0'], 3108: ['1-0', '2-0', '5-0', '3-0', '4-0'], 3109: ['1-0', '2-0', '5-1', '3-0', '-1-0'], 3110: ['4-0', '2-0', '5-1', '1-0', '3-0'], 3111: ['4-0', '2-0', '5-1', '1-0', '3-1'], 3112: ['4-0', '2-1', '1-0', '3-0', '5-0']})


defaultdict(list,
            {3100: ['2-0', '1-0', '3-0', '4-0', '5-0'],
             3101: ['2-0', '1-0', '3-0', '5-0', '4-0'],
             3102: ['2-0', '1-0', '5-0', '4-0', '3-0'],
             3103: ['2-0', '1-0', '5-0', '4-0', '3-0'],
             3104: ['2-1', '1-1', '4-0', '5-0', '3-0'],
             3105: ['2-1', '1-1', '5-0', '3-0', '4-0'],
             3106: ['2-1', '1-1', '3-0', '4-0', '5-0'],
             3107: ['2-1', '1-1', '3-0', '5-0', '4-0'],
             3108: ['1-0', '2-0', '5-0', '3-0', '4-0'],
             3109: ['1-0', '2-0', '5-1', '3-0', '-1-0'],
             3110: ['4-0', '2-0', '5-1', '1-0', '3-0'],
             3111: ['4-0', '2-0', '5-1', '1-0', '3-1'],
             3112: ['4-0', '2-1', '1-0', '3-0', '5-0']})

In [14]:
# test : q_students_pass1
test_intervenants_data = pd.DataFrame([
  [],
  [
    { "id" : 10, "batch_size":3 },
    { "id" : 11, "batch_size":3 }
  ],
    [
    { "id" : 20, "batch_size":3 },
    { "id" : 21, "batch_size":3 }
  ],
  [
    { "id" : 30, "batch_size":3 }  
  ]
])
test_students = [ 
    { "id" : 3100, "wishes" : [ 1,2,3 ], 'itv_visited' : []  }, 
    { "id" : 3101, "wishes" : [ 1,2,3 ], 'itv_visited' : []  }, 
    { "id" : 3102, "wishes" : [ 1,2,3 ], 'itv_visited' : []  }, 
    { "id" : 3103, "wishes" : [ 1,2,3 ], 'itv_visited' : []  },
    { "id" : 3104, "wishes" : [ 1,2,3 ], 'itv_visited' : []  }, 
    { "id" : 3105, "wishes" : [ 1,2,3 ], 'itv_visited' : []  }, 
]

NB_SLOTS = 3

timetable,_ = get_timetable_student_driven(NB_SLOTS, test_students, test_intervenants_data)
print(pd.DataFrame(timetable))

                     0                    1                    2  \
0  (3100, (0, (1, 0)))  (3101, (0, (1, 0)))  (3102, (0, (1, 0)))   
1  (3100, (1, (2, 0)))  (3101, (1, (2, 0)))  (3102, (1, (2, 0)))   
2  (3100, (2, (3, 0)))  (3101, (2, (3, 0)))  (3102, (2, (3, 0)))   

                      3                     4                     5  
0   (3103, (0, (1, 1)))   (3104, (0, (1, 1)))   (3105, (0, (1, 1)))  
1   (3103, (1, (2, 1)))   (3104, (1, (2, 1)))   (3105, (1, (2, 1)))  
2  (3103, (2, (-1, 0)))  (3104, (2, (-1, 0)))  (3105, (2, (-1, 0)))  


## Tests

### Student driven

In [15]:
test_intervenants_data = pd.DataFrame([
  [],
  [ { "id" : 10, "batch_size":3 }, { "id" : 11, "batch_size":2 } ],
  [ { "id" : 20, "batch_size":3 }, { "id" : 21, "batch_size":2 } ],
  [ { "id" : 30, "batch_size":2 } ],
  [ { "id" : 40, "batch_size":1 } ],
  [ { "id" : 50, "batch_size":1 } ]
])

test_students = [ 
    { "id" : 3100, "wishes" : [ 2,1,3 ], 'itv_visited' : [] }, 
    { "id" : 3101, "wishes" : [ 2,1,3 ], 'itv_visited' : []  }, 
    { "id" : 3102, "wishes" : [ 2,1,4 ], 'itv_visited' : []  }, 
    { "id" : 3103, "wishes" : [ 2,1,4 ], 'itv_visited' : []  },
    { "id" : 3104, "wishes" : [ 2,1,4 ], 'itv_visited' : []  }, 
    { "id" : 3105, "wishes" : [ 2,1,4 ], 'itv_visited' : []  }, 
    { "id" : 3106, "wishes" : [ 2,1,3 ], 'itv_visited' : []  }, 
    { "id" : 3107, "wishes" : [ 1,2,3 ], 'itv_visited' : []  },
    { "id" : 3108, "wishes" : [ 1,2,3 ], 'itv_visited' : []  }, 
    { "id" : 3109, "wishes" : [ 1,2,3 ], 'itv_visited' : []  }, 
    { "id" : 3110, "wishes" : [ 1,2,3 ], 'itv_visited' : []  }, 
    { "id" : 3111, "wishes" : [ 1,2,4 ], 'itv_visited' : []  },
    { "id" : 3112, "wishes" : [ 1,3,4 ], 'itv_visited' : []  }
]

NB_SLOTS=3
timetable, test_students = get_timetable_student_driven(NB_SLOTS, test_students, test_intervenants_data)
pd.DataFrame(timetable)


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,"(3100, (0, (2, 0)))","(3101, (0, (2, 0)))","(3102, (0, (2, 0)))","(3103, (0, (2, 1)))","(3104, (0, (2, 1)))","(3105, (0, (4, 0)))","(3106, (0, (3, 0)))","(3107, (0, (1, 0)))","(3108, (0, (1, 0)))","(3109, (0, (1, 0)))","(3110, (0, (1, 1)))","(3111, (0, (1, 1)))","(3112, (0, (3, 0)))"
1,"(3100, (1, (1, 0)))","(3101, (1, (1, 0)))","(3102, (1, (1, 0)))","(3103, (1, (1, 1)))","(3104, (1, (1, 1)))","(3105, (1, (2, 0)))","(3106, (1, (2, 0)))","(3107, (1, (2, 0)))","(3108, (1, (2, 1)))","(3109, (1, (2, 1)))","(3110, (1, (3, 0)))","(3111, (1, (4, 0)))","(3112, (1, (-1, 0)))"
2,"(3100, (2, (3, 0)))","(3101, (2, (3, 0)))","(3102, (2, (4, 0)))","(3103, (2, (-1, 0)))","(3104, (2, (-1, 0)))","(3105, (2, (1, 0)))","(3106, (2, (1, 0)))","(3107, (2, (-1, 0)))","(3108, (2, (-1, 0)))","(3109, (2, (-1, 0)))","(3110, (2, (2, 0)))","(3111, (2, (2, 0)))","(3112, (2, (1, 0)))"


In [16]:
test_q_intervenants = init_q_intervenants(test_intervenants_data)

print(get_evaluation(timetable, test_students, test_q_intervenants))

overall_student_satisfaction : 0.846
overall ratio visited 1.0
nb_slots : [0, 1, 2]
nb_no_visits_itv : 5
None


In [17]:
timetable

[[(3100, (0, (2, 0))),
  (3101, (0, (2, 0))),
  (3102, (0, (2, 0))),
  (3103, (0, (2, 1))),
  (3104, (0, (2, 1))),
  (3105, (0, (4, 0))),
  (3106, (0, (3, 0))),
  (3107, (0, (1, 0))),
  (3108, (0, (1, 0))),
  (3109, (0, (1, 0))),
  (3110, (0, (1, 1))),
  (3111, (0, (1, 1))),
  (3112, (0, (3, 0)))],
 [(3100, (1, (1, 0))),
  (3101, (1, (1, 0))),
  (3102, (1, (1, 0))),
  (3103, (1, (1, 1))),
  (3104, (1, (1, 1))),
  (3105, (1, (2, 0))),
  (3106, (1, (2, 0))),
  (3107, (1, (2, 0))),
  (3108, (1, (2, 1))),
  (3109, (1, (2, 1))),
  (3110, (1, (3, 0))),
  (3111, (1, (4, 0))),
  (3112, (1, (-1, 0)))],
 [(3100, (2, (3, 0))),
  (3101, (2, (3, 0))),
  (3102, (2, (4, 0))),
  (3103, (2, (-1, 0))),
  (3104, (2, (-1, 0))),
  (3105, (2, (1, 0))),
  (3106, (2, (1, 0))),
  (3107, (2, (-1, 0))),
  (3108, (2, (-1, 0))),
  (3109, (2, (-1, 0))),
  (3110, (2, (2, 0))),
  (3111, (2, (2, 0))),
  (3112, (2, (1, 0)))]]

In [18]:
test_q_intervenants = init_q_intervenants(test_intervenants_data)

pd.DataFrame(get_timetable_intervenants(timetable, test_q_intervenants))
#get_timetable_intervenants(timetable, test_intervenants_data)

nb_slots : [0, 1, 2]


,10,11,20,21,30,40,50
0,3,2,3,2,2,1,0
1,3,2,3,2,1,1,0
2,3,0,2,0,2,1,0


In [19]:
timetable_student_view(timetable)

defaultdict(<class 'list'>, {3100: ['2-0', '1-0', '3-0'], 3101: ['2-0', '1-0', '3-0'], 3102: ['2-0', '1-0', '4-0'], 3103: ['2-1', '1-1', '-1-0'], 3104: ['2-1', '1-1', '-1-0'], 3105: ['4-0', '2-0', '1-0'], 3106: ['3-0', '2-0', '1-0'], 3107: ['1-0', '2-0', '-1-0'], 3108: ['1-0', '2-1', '-1-0'], 3109: ['1-0', '2-1', '-1-0'], 3110: ['1-1', '3-0', '2-0'], 3111: ['1-1', '4-0', '2-0'], 3112: ['3-0', '-1-0', '1-0']})


defaultdict(list,
            {3100: ['2-0', '1-0', '3-0'],
             3101: ['2-0', '1-0', '3-0'],
             3102: ['2-0', '1-0', '4-0'],
             3103: ['2-1', '1-1', '-1-0'],
             3104: ['2-1', '1-1', '-1-0'],
             3105: ['4-0', '2-0', '1-0'],
             3106: ['3-0', '2-0', '1-0'],
             3107: ['1-0', '2-0', '-1-0'],
             3108: ['1-0', '2-1', '-1-0'],
             3109: ['1-0', '2-1', '-1-0'],
             3110: ['1-1', '3-0', '2-0'],
             3111: ['1-1', '4-0', '2-0'],
             3112: ['3-0', '-1-0', '1-0']})

In [20]:
pd.DataFrame(timetable_student_view(timetable)).T

defaultdict(<class 'list'>, {3100: ['2-0', '1-0', '3-0'], 3101: ['2-0', '1-0', '3-0'], 3102: ['2-0', '1-0', '4-0'], 3103: ['2-1', '1-1', '-1-0'], 3104: ['2-1', '1-1', '-1-0'], 3105: ['4-0', '2-0', '1-0'], 3106: ['3-0', '2-0', '1-0'], 3107: ['1-0', '2-0', '-1-0'], 3108: ['1-0', '2-1', '-1-0'], 3109: ['1-0', '2-1', '-1-0'], 3110: ['1-1', '3-0', '2-0'], 3111: ['1-1', '4-0', '2-0'], 3112: ['3-0', '-1-0', '1-0']})


,0,1,2
3100,2-0,1-0,3-0
3101,2-0,1-0,3-0
3102,2-0,1-0,4-0
3103,2-1,1-1,-1-0
3104,2-1,1-1,-1-0
3105,4-0,2-0,1-0
3106,3-0,2-0,1-0
3107,1-0,2-0,-1-0
3108,1-0,2-1,-1-0
3109,1-0,2-1,-1-0


## Tests données réelles

### Student driven

In [21]:
test_intervenants_data = pd.read_json('intervenants_data.json')

# importation d'un échantillon de données pour les tests unitaires
students_2025 = pd.read_json('data-2025.json')
result = students_2025.to_json(orient="index")
parsed = loads(result)
test_students_2025 = list(parsed.values())

In [22]:

NB_SLOTS = 6
timetable, test_students_2025 = get_timetable_student_driven(NB_SLOTS, test_students_2025, test_intervenants_data)
#timetable


In [23]:

timetable = convert_to_student_timetable(NB_SLOTS, test_students_2025)
timetable

[[(3100, (0, (8, 0))),
  (3101, (0, (3, 0))),
  (3102, (0, (9, 0))),
  (3103, (0, (17, 0))),
  (3104, (0, (10, 0))),
  (3105, (0, (6, 0))),
  (3106, (0, (9, 0))),
  (3107, (0, (9, 0))),
  (3108, (0, (11, 0))),
  (3109, (0, (1, 0))),
  (3110, (0, (1, 0))),
  (3111, (0, (10, 0))),
  (3112, (0, (8, 0))),
  (3113, (0, (7, 0))),
  (3114, (0, (14, 0))),
  (3115, (0, (4, 0))),
  (3116, (0, (8, 0))),
  (3117, (0, (5, 0))),
  (3118, (0, (13, 0))),
  (3119, (0, (7, 0))),
  (3120, (0, (7, 0))),
  (3121, (0, (8, 0))),
  (3122, (0, (18, 0))),
  (3123, (0, (9, 0))),
  (3124, (0, (11, 0))),
  (3125, (0, (11, 0))),
  (3126, (0, (16, 0))),
  (3127, (0, (16, 0))),
  (3200, (0, (18, 0))),
  (3201, (0, (1, 0))),
  (3202, (0, (10, 0))),
  (3203, (0, (1, 0))),
  (3204, (0, (14, 0))),
  (3205, (0, (10, 0))),
  (3206, (0, (1, 1))),
  (3207, (0, (20, 0))),
  (3208, (0, (1, 1))),
  (3209, (0, (8, 1))),
  (3210, (0, (4, 0))),
  (3211, (0, (18, 0))),
  (3212, (0, (10, 1))),
  (3213, (0, (12, 0))),
  (3214, (0, (9

In [24]:
q_intervenants = init_q_intervenants(test_intervenants_data)

pd.set_option("display.max_columns", 65)
pd.set_option("display.max_rows", 6)
pd.DataFrame(get_timetable_intervenants(timetable, q_intervenants))
#get_timetable_intervenants(timetable, test_intervenants_data)

nb_slots : [0, 1, 2, 3, 4, 5]


,10,11,12,20,21,30,31,40,50,51,52,53,54,55,60,61,62,63,70,80,81,82,83,90,91,92,93,94,95,100,101,102,110,111,112,113,114,120,130,131,140,141,142,150,151,160,161,162,163,170,171,172,180,181,182,190,191,192,200,210,211
0,4,4,4,4,4,4,4,4,4,4,4,4,4,2,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,3,0,4,4,4,4,4,4,4,2,4,4,3,0,4,2,0,4,4,4,4,3,0,4,4,4
1,4,4,4,4,4,4,4,4,4,4,3,0,0,0,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,0,4,4,4,4,4,4,4,0,4,4,4,2,4,0,0,4,4,4,4,1,0,4,4,3
2,4,4,4,4,4,4,4,4,4,4,3,0,0,0,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,3,0,0,4,4,1,4,4,4,4,1,4,2,0,0,4,0,0,4,4,4,4,4,3,4,4,0
3,4,4,4,4,4,4,4,4,4,3,0,0,0,0,4,4,4,4,4,4,4,4,4,4,4,4,4,2,0,4,4,4,3,0,0,0,0,4,4,3,4,4,4,4,2,4,4,1,0,4,0,0,4,4,3,4,4,0,4,4,1
4,4,4,4,4,4,4,1,4,4,0,0,0,0,0,4,4,4,4,4,4,4,4,4,4,4,4,4,0,0,4,4,4,4,4,0,0,0,4,2,0,4,4,4,2,0,4,4,0,0,3,0,0,4,0,0,2,0,0,4,1,0
5,4,4,4,4,4,0,0,4,0,0,0,0,0,0,4,4,4,4,4,4,4,1,0,0,0,0,0,0,0,4,4,4,0,0,0,0,0,4,0,0,4,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,4,0,0


In [25]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 240)
timetable_students = pd.DataFrame(timetable_student_view(timetable)).T
timetable_students
timetable_students.to_csv('output_timetable_students_std_driven.csv', sep=';', header=False)

defaultdict(<class 'list'>, {3100: ['8-0', '3-0', '7-0', '6-0', '9-0', '-1-0'], 3101: ['3-0', '7-0', '6-0', '19-0', '11-0', '-1-0'], 3102: ['9-0', '10-0', '16-0', '6-0', '11-0', '-1-0'], 3103: ['17-0', '16-0', '7-0', '8-0', '9-0', '-1-0'], 3104: ['10-0', '11-0', '12-0', '16-0', '2-0', '-1-0'], 3105: ['6-0', '10-0', '11-0', '12-0', '3-0', '-1-0'], 3106: ['9-0', '10-0', '8-0', '20-0', '16-0', '-1-0'], 3107: ['9-0', '20-0', '6-0', '16-0', '11-0', '-1-0'], 3108: ['11-0', '10-0', '13-0', '3-0', '7-0', '-1-0'], 3109: ['1-0', '2-0', '3-0', '4-0', '5-0', '-1-0'], 3110: ['1-0', '18-0', '4-0', '2-0', '9-0', '-1-0'], 3111: ['10-0', '5-0', '18-0', '8-0', '9-0', '-1-0'], 3112: ['8-0', '6-0', '4-0', '5-0', '9-1', '-1-0'], 3113: ['7-0', '20-0', '6-0', '3-0', '8-0', '-1-0'], 3114: ['14-0', '9-0', '2-0', '21-0', '8-0', '-1-0'], 3115: ['4-0', '5-0', '16-0', '9-0', '20-0', '-1-0'], 3116: ['8-0', '10-1', '6-0', '9-0', '16-0', '-1-0'], 3117: ['5-0', '6-0', '20-0', '9-0', '14-0', '-1-0'], 3118: ['13-0', '21